# Predicting *sentimental impact*

For this project, we decided to try and predict the *sentimental impact* certain conflicts have in other countries. Bear in mind what you are about to read is but a sequel to the [project's root README](https://github.com/nunomota/ada2017-hw/blob/master/project/README.md) - so that we can keep information duplication to a bare minimum.

Although it may seem quite complicated, this end goal can be broken down into smaller steps:

1. [Language Recognition](https://github.com/nunomota/ada2017-hw/blob/master/project/scripts/language_recognition.py): To identify and keep only English Tweets

2. [Name Entity Recognition](https://github.com/nunomota/ada2017-hw/blob/master/project/scripts/name_entity_recognition.py): To find which country a Tweet might be referring to

3. [Sentiment Analysis](https://github.com/nunomota/ada2017-hw/blob/master/project/scripts/sentiment_analysis.py): To measure the sentiment associated with a Tweet

4. Location and textual data: Taken from UCDP and Twitter datasets

**Note:** To keep the code clear and concise, we moved the code related to these steps into separate scripts. For the most part, this notebook covers a higher level of the implementation - if you would like to know more about the specifics, everything is properly commented within each script.

## Language Recognition (LR)

The [language recognition script](https://github.com/nunomota/ada2017-hw/blob/master/project/scripts/language_recognition.py) simply makes use of the [langdetect library](https://pypi.python.org/pypi/langdetect) to output whether or not a certain Tweet is written in English. Although this library's reliability increses with the text's length - and hence a bit limited for Tweets - it tends to give more *false negatives* than *false positives*. This is not a problem since we have a large amount of data and we would rather discard some English Tweets than keeping others in foreign languages.

## Name Entity Recognition (NER)

This is the trickiest part of the project. The main goal is to find whether or not a Tweet is talking about a country. For this part, we opted to use a natural language processing library called [Spacy](https://spacy.io/), for a couple of reasons:

1. Simple text comparison tends to over-identify entities in a text (loose words might match but, contextually, they might not be referring to a place)
2. Other alternatives, like [NLTK](http://www.nltk.org/) tended to be slower
3. Spacy comes with built-in models trained with [OntoNotes 5](https://catalog.ldc.upenn.edu/ldc2013t19), giving us a wide range of entity labels to work with

Although it seems to work quite well, we noticed many cases where it may (or not) consider certain entities based on their capitalization alone. To minimize these errors, we created a [Tweet Generator](https://github.com/nunomota/ada2017-hw/blob/master/project/tweets_generator.ipynb). This notebook creates Markov Chains models by analysis several corpus we provide (stories, geo-political or religious texts, news, etc...) and, through the [markovify library](https://github.com/jsvine/markovify), it generates several texts with specified length (in this case 140, which is a Tweet's max length). The aim is to, as a next step, go through its interactive generator and manually specify what we find in the generated text. From the overall output of that process we expect to find which text-formatting techniques provide a higher accuracy for our NER model (e.g. lower-case text for cities, upper-case for countries or title-case for religions).

We also enriched our data with a lot of other datasets (also specified in the [project's root README](https://github.com/nunomota/ada2017-hw/blob/master/project/README.md)), creating bi-lateral associations between:

* Countries
* Cities
* Nationalities
* Religions
* Religious affiliations
* Currencies

Also, we don't just keep words that directly refer to a country. In the context of *religions*, *cities* and *currencies* there are a lot of possible countries related to a single word (e.g. several countries might have a city with the same name). In these cases, we take into account the probability each entity is referring to each country. We then take all of the entites, analyze the probabilities and find which are the *most likely* countries. If a country is within that set of countries and its probability is above a certain threshold, we assume the Tweet is referring to it.

## Sentiment Analysis (SA)

The [sentiemnt analysis script](https://github.com/nunomota/ada2017-hw/blob/master/project/scripts/sentiment_analysis.py) depends on [NLTK's Vader](http://www.nltk.org/_modules/nltk/sentiment/vader.html) sentiment analyzer. From the final result of this library, we then return the *compound* value (which already takes into account the *positive*, *negative* and *neutral* sentiments in the text) and will be within the range [-1.0, 1.0].

## Sentimental impact

We will then associate the *SA* value with each and every Tweet that passed through our *LR* and *NER* filtering. Bear in mind that for any given conflict, before we even apply one of the above filters, we discard Tweets outside a certain time-frame - around a conflict's start date. We then do a daily weighting of the *absolute* value of the sentiments (penalizing outliers, to eliminate random over-emotional tweets).

To define *sentimental impact* we analyse the contrast between the *before* and *after* of the daily sentimental averages (not penalizing outliers, since one-day conflicts might have an emotional peak for a single day and they still might be as *emotionaly impactful* as a weekly conflict). Following these guidelines, we still have to chose the most appropriate measure to use in both *daily* and *before/after* cases.

## Next steps

Although we already extracted the information we need from the UCDP dataset, we didn't do it yet with the Twitter one. Since we didn't have access to it from the start, we decided to focus on the above topics in the mean time - using a local, smaller dataset. Because of this, our next steps will be:

1. Categorize conflicts (e.g. according to casualties)
2. Filter Tweets that are within 3 days (before and after) each of the conflicts' start dates
3. Filter Tweets with *LR* and *NER* 
4. Apply *SA* on the remaining Tweets and calculate *sentimental impact*
5. Train a machine learning model that uses *conflict category* and *conflict country* as features and tries to predict *sentiment impact*
6. Show predictions accross countries for same-category conflicts

In [2]:
# Add scripts folder to path
import sys
sys.path.insert(0, 'scripts')

In [3]:
# Import necessary modules for Tweet analysis
import name_entity_recognition as ner
import sentiment_analysis as sa
import language_recognition as lr

In [4]:
samples = [
    'Some random sentence about a country',
    'I am talking about Portugal',
    'Portuguese people always arrive late',
    'John doesn\'t think Spain is good for vacation',
    'This was easier than I thought...',
    'Now a Saudi Arabia comment',
    'With a Saudi Arabian nationality notation, along with religious christianity references'
]

In [9]:
# TODO fix according to new country_city dataframe
ner.is_tweet_about_country(samples[2], 'PT')


Portuguese -> NORP
NORP -> Nationality!
{'PT': 1.0}


True

In [10]:
sa.get_sentiment_intensity(samples[2])

0.0

In [11]:
lr.is_tweet_english(samples[2])

True